# Data preprocessing: more art than science?
### Written by Nadia Blostein

## Contents of this notebook:
<ol>
<li>Load and examine your data</li>
<li>Data reformatting</li>
<li>Data filtering</li>
<li>Data transforms</li>
<li>Data visualization</li>
<li>Examining 2D images</li>
</ol>

# Setup
Fetch the dataset that you'll be working with throughout this assignment.


In [ ]:
!git clone https://github.com/NadiaBlostein/Open-Access-HCP-Data.git

Examine your directory structure with the `ls` command. To invoke this command from a jupyter notebook in Google Colab, the `ls` command should be preceded with `!`. 

In [ ]:
!ls

Change the working directory of the notebook to within the folder `Open-Access-HCP-Data`. The `cd` command should be preceded with `%`.


In [ ]:
%cd Open-Access-HCP-Data 
!ls

In this assignment, we will be working with data from the Human Connectome Project (HCP). You can read more about the data [here](https://github.com/NadiaBlostein/McMedHacks2022_Prep_Week_3_Assignment#readme). Specifically, you will preprocess `.csv` (in the `HCP_csv_data` folder) and `.png` files (in the `HCP_2D_slices_MRI_data` folder).

In [ ]:
!ls HCP_2D_slices_MRI_data

In [ ]:
!ls HCP_csv_data 

# 1. Load and examine your data

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("HCP_csv_data/unrestricted_HCP_behavioral.csv")

**Question 1:** How many subjects (rows) and features (columns) do you have?\
_PS: You cannot always assume that your subjects are rows and features are columns, and sometimes there may be strange header rows with useless information that you need to remove._

In [ ]:
# SOLUTION
print(f"Num subjects: {df.shape[0]}")
print(f"Num features: {df.shape[1]}")

Feel free to examine your data frame in the empty code cell below. Examples of valuable commands for a preliminary look at one's data frame: `df.info()`, `df.describe()`, `df.columns()`, `df.head()`, `df.tail`.

Next, we will select the `Gender` column to count how many instances we have of each value.

In [ ]:
print(f"unique values of column: {df['Gender'].unique()}") # prints list of unique values in a feature / column
print(f"\nvalue counts of column:\n{df['Gender'].value_counts()}") # counts how many times each unique value in a feature / column occurs

**Question 2:** Is there another way to count how many males and females are in this dataset?

In [ ]:
# SOLUTION
print(f"Number females: {len(df[df['Gender']=='F'])}")
print(f"Number males: {len(df[df['Gender']=='M'])}")

**Question 3:** Display only the rows associated with female subjects.

In [ ]:
# SOLUTION
df[df['Gender']=="F"]

### A side note on documentation
Woah! So many columns and abbreviations! What do they all mean? Make sure you know where your [dataset's documentation](https://wiki.humanconnectome.org/display/PublicData/HCP-YA+Data+Dictionary-+Updated+for+the+1200+Subject+Release#HCPYADataDictionaryUpdatedforthe1200SubjectRelease-Instrument:Demographics) is.

Unfortunately, thorough documentation is not always available. Some data types are also very field-specific and require the help of experts, which is part of what makes machine learning so wonderfully interdisciplinary.

Let's take a look at what features we have here:

In [ ]:
print(df.columns)

**Question 4:** The script above is not printing ALL of the columns... How do we fix that to be able to see all the features in the dataset?

In [ ]:
# SOLUTION
for col in df.columns: print(col)

# 2. Data reformatting

## 2.1 Selecting the features you want to work with:
As you saw above, this dataframe has 499 features. Often, you will only want to work with a subset of the features of a dataframe, so you will have to create a new dataframe with this subset.

In [ ]:
basics = ['Subject','Gender','Age','PSQI_BedTime']
df[basics]

Let's also add all of some cognitive variables to the mix! Specifically, we'll select the measures related to fluid intelligence (they start with `PMAT` for Penn Matrix Test) and impulsivity (they start with `DDisc` for Delay Discounting).\
**Question 5:** Fill in the missing code below.

In [ ]:
cognition = ['Subject','Gender','Age','PSQI_BedTime']
for col in df.columns:
    if (col.find("PMAT")!=-1 or col.find("DDisc")!=-1):
        # FILL IN THE BLANK. SOLUTION: cognition.append(col)
        cognition.append(col)
print(f"List of variables we will be looking at: {cognition}") # PS: f-strings will be very useful for you in your Python journey!

**Question 6:** Now that we made a list of all of the features we want to examine, select this subset of our data and make it a separate dataframe called `df_cognition`.

In [ ]:
# SOLUTION
df_cognition = df[cognition]
df_cognition.head()

Quick sanity check to make sure everything is behaving as expected:

In [ ]:
df_cognition.shape

## 2.2 Merging two dataframes

For our dataset of 1206 subjects, we have information about their gender, age range and a variety of cognitive measures. It would be interesting to integrate some other data as well. You have been provided with a separate file that contains brain structure volume data obtained from the neuroimaging data of these same subjects (note that not all subjects in the HCP have neuroimaging data) (more information about how these volumes were obtained can be found [here](https://github.com/NadiaBlostein/Open-Access-HCP-Data#readme).

In [ ]:
df_volumes = pd.read_csv("HCP_csv_data/HCP_volumes.csv")

**Question 7:** How many subjects and features does `df_volumes` have?

In [ ]:
# SOLUTION
print(f"Num subjects: {df_volumes.shape[0]}")
print(f"Num features: {df_volumes.shape[1]}")

**Question 8:** Print the mean and standard deviation of total brain volume (TBV) of this sample. Note that the unit is in mm$^{3}$.

In [ ]:
# SOLUTION
print(f"Mean TBV: {df_volumes['TBV'].mean()} mm3")
print(f"TBV standard deviation: {df_volumes['TBV'].std()} mm3")

**Question 9:** Can you round it to two decimal places?

In [ ]:
# SOLUTION
print(f"Mean TBV: {round(df_volumes['TBV'].mean(),2)} mm3")
print(f"TBV standard deviation: {round(df_volumes['TBV'].std(),2)} mm3")

**Question 10:** List all the subjects whose TBV is above average.

In [ ]:
# SOLUTION
df_volumes[df_volumes['TBV']>df_volumes['TBV'].mean()]

Ok! Let's merge our dataframes. One problem is that our behavioral data has 1206 subjects and our volume data has 1086 subjects. 

**Question 11:** Create a new dataframe called `df_final` where we only keep the subjects for which we have all of our features.

In [ ]:
# SOLUTION
df_final=pd.merge(df_cognition,df_volumes, on='Subject')
df_final.head()

Quick sanity check to make sure everything is behaving as expected:

In [ ]:
print(df_final.shape)

Mission accomplished! We have 1086 and 35 features. However, we would normally expect 36 features (22 features from `df_cognition` and 14 features from `df_volumes`).\
**Question 12:** Why do we only have 35 features in `df_final`?

SOLUTION\
Because `df_cognition` and `df_volumes` have 1 column in common, `Subject`. In fact, we already told the pandas `merge` method to merge the two dataframes along this shared column. Luckily, this method is smart enough not to repeat a shared column in the new dataframe!

# 3. Data filtering

## 3.1 Removing features that you do not need

**Question 13:** Complete the following code in order to find and remove any duplicate columns.

In [ ]:
cols_to_drop=[]
for i in range(df_final.shape[1]):
    for j in range(i+1,df_final.shape[1]):
        col1=df_final.columns[i]
        col2=df_final.columns[j]
        if (df_final[col1].equals(df_final[col2])):
            print(f"Duplicate columns: {col1, col2}")
            cols_to_drop.append(col2)
# FILL IN THE BLANK. SOLUTION: df_final.drop(cols_to_drop, inplace=True, axis=1)

Quick sanity check to make sure everything is behaving as expected:

In [ ]:
print(df_final.shape) 

# 3.2 Making your data machine-readable

To be machine-readable, your variables need to be numerical. Want to check?

In [ ]:
df_final.dtypes

We have 3 columns that are non-numerical (meaning that they are neither floats nor integers): `Gender`,`Age`,`PSQI_BedTime`. Let's figure out how to handle them, one at a time.

### One-hot encoding or binarizing your data
First, we know that the `Gender` column is categorical and has two unique values: `M` or `F`. 

**Question 14:** Replace all of your `M` values with 1 and `F` values with 2. Hint: `.replace()` can be quite helpful. \
_PS: Make sure to SAVE your data preprocessing code. If ever you forget what number you assigned to what value in the `Gender` category, you will always be able to look back at your code!_

In [ ]:
# SOLUTION
df_final['Gender'] = df_final['Gender'].replace('M',1)
df_final['Gender'] = df_final['Gender'].replace('F',2)

In [ ]:
df_final

However, suppose that you actually had more than 2 numerical values for this feature (e.g. `M`,`F`,`other`). If you just convert categorical variables to numerical values (ex: `M`=1,`F`=2,`other`=3), you give a "distance" to the relationship between variables. For instance, since 1 is closer to 2 than to 3, you are telling your machine that `M` is "closer" to `F` (`distance = 2 - 1 = 1`) than to `other` (`distance = 3 - 1 = 2`). We want our categories to be independent. That's where one hot encoding comes into play: "[A representation of categorical variables as binary vectors](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/)."

Let's revert our `Gender` feature back to its original form, and see how to one-hot encode it.

In [ ]:
df_final['Gender'] = df_final['Gender'].replace(1,'M')
df_final['Gender'] = df_final['Gender'].replace(2,'F')

one_hot = pd.get_dummies(df_final['Gender'])
one_hot

As you can see, we convert 1 categorical feature into N binary features, where N = number of values that your feature can take on (2 in this example).\
**Question 15:** Add `one_hot` to `df_final` and make sure to drop the `Gender` column (to avoid duplicate information).\

In [ ]:
# SOLUTION
df_final = df_final.join(one_hot)
df_final = df_final.drop('Gender',axis = 1)

Notice that you need to reassign your changes to `df_final` (`df_final = `), or they will not get saved. Quick sanity check to make sure everything is behaving as expected:

In [ ]:
df_final

### Parsing strings in your data

#### Handling the `Age` feature

Remember that our `Age` feature is also non-numerical!

In [ ]:
df_final['Age'].value_counts()

**Question 16:** For the sake of this exercise, replace 36+ with a range of 36-40.

In [ ]:
# SOLUTION
df_final['Age'] = df_final['Age'].replace("36+",'36-40')

Quick sanity check to make sure everything is behaving as expected:

In [ ]:
df_final['Age'].value_counts()

Our age range values are organized very clearly: `minimum age–maximum age`. These age range strings can therefore be split around the `-` such that you create two new columns: one for `minimum age` and one for `maximum age`.

In [ ]:
fix_age = df_final['Age'].str.split('-', 1, expand=True)
fix_age.columns = ['min','max']
fix_age

**Question 17:** In `df_final`, replace the age range string (from the `Age` columne) for each subject with the mean of the subject's respective age range. _Hint: make sure that the `min` and `max` columns of the `fix_age` df are converted to a numberical data type._

In [ ]:
# SOLUTION
fix_age["min"] = fix_age['min'].astype(float)
fix_age["max"] = fix_age['max'].astype(float)
fix_age['mean'] = (fix_age['max']+fix_age['min'])/2
df_final['Age'] = fix_age['mean']

In [ ]:
df_final.head(3)

#### Handling the `PSQI_BedTime` feature

Convert your bed time variable from HH:MM:SS to seconds! The next line of code uses some slightly more advanced functions that will not be covered today, but you can walk through it step by step to figure out what each method does.

In [ ]:
ftr = [3600,60,1]
for i in range(len(df_final['PSQI_BedTime'])):
    x = sum([a*b for a,b in zip(ftr, map(int,df_final['PSQI_BedTime'][i].split(':')))])
    df_final['PSQI_BedTime'][i] = x

#### A note on other strings that often crop up in dataframes and need to be replaced with numbers!
`df_final = df_final.replace('FALSE',0)` \
`df_final = df_final.replace('TRUE',1)` \
`df_final = df_final.replace(False,0)` \
`df_final = df_final.replace(True,1)` \
`df_final = df_final.replace('0',0)` # example of random spaces \
`df_final = df_final.replace(' ',np.NaN)` # example of random spaces

**Question 18:** Write a quick line of code that makes sure that every column of `df_final` is of type float!

In [ ]:
# SOLUTION
for col in df_final.columns: df_final[col] = df_final[col].astype(float)

## 3.3 Handling not available (NA) and inf data:

Sometimes, Python will convert some of your values to + or - infinity, which will result in downstream errors. Convert them to NA, and then handle them as NA values.

In [ ]:
df_final = df_final.replace([np.inf, -np.inf], np.nan)

Next, you need to deal with your NA values. 

**Question 19:** How many nas do you have?

In [ ]:
# SOLUTION
df_final.isna().sum()

There is a variety of ways to handle NA data. The most simple approach is to replace NA data with the median (or mean) value of the feature of interest. There are [other](https://towardsdatascience.com/6-different-ways-to-compensate-for-missing-values-data-imputation-with-examples-6022d9ca0779) [more](https://arxiv.org/abs/1804.11087) sophisticated data imputation techniques out there, many of which actually leverage machine learning tools (so meta)!

However, if a feature has too many NAs, you may want to remove it completely. Define a threshold for the minimal number of missing values that qualifies a feature for removal from your dataset. Here, 12 is quite stringent.

**Question 20:** Fill in the blank line of the code below.

In [ ]:
threshold=12

remove_cols = []
for i in range(len(df_final.columns)):
    if (df_final.iloc[:,i].isnull().sum() >= threshold):
        remove_cols.append(df_final.columns[i])
# FILL IN THE BLANK. SOLUTION: df_final = df_final.drop(columns=remove_cols)

**Question 21:** Replace the NA values of this dataframe with the feature-specific median (the median is more robust against outliers than the mean is).

In [ ]:
# SOLUTION
for col in df_final.columns: df_final[col].fillna(df_final[col].median(), inplace=True)

Quick sanity check to make sure everything is behaving as expected:

In [ ]:
df_final.isna().sum().sum()
# note the difference between df_final.isna().sum() and df_final.isna().sum().sum()

## 3.4 Removing columns with a standard deviation of 0

**Question 22:** Write a line of code that removes all columns with a standrard deviation of 0. Why do you think we would want to do this?

In [ ]:
# SOLUTION
df_final = df_final.loc[:, df_final.std() > 0]

Quick sanity check to make sure everything is behaving as expected:

In [ ]:
for col in df_final.columns:
    if (df_final[col].std()==0): print(col)

# 4. Data transforms

You usually need to perform some sort of feature scaling /normalization to make sure that all of your variables are in the same range (this affects gradient-descent-based algorithms and distance-based algorithms).

**Min-Max Scaling / Normalization:** X' = (X-Xmin) / (Xmax-Xmin), X' always ends up with a range of \[0,1\] \
**Standardization / Standard Scaler / Z-score):** X' = (X-$\mu$)/$\sigma$

Which to use? Depends on your data! _"Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution"_ (from [here](https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/)). Other [popular scaling techniques](https://www.analyticsvidhya.com/blog/2020/07/types-of-feature-transformation-and-scaling/) include the log transform (you often see this with GWAS, ie genome-wide association studies) and dividing your column-wise values by the absolute value of the maximal value of each column (max abs scaler).

Example 1: Min-Max Scaling

In [ ]:
minMaxScaled_df_final=(df_final-df_final.min())/(df_final.max()-df_final.min())

Example 2: Standard Scaling\
**Question 23:** Implement a line of code that performs the following column-wise z-scoring (X' = (X-$\mu$)/$\sigma$).

In [ ]:
# SOLUTION
standardized_df_final=(df_final-df_final.mean())/df_final.std()

Example 3: Sklearn Min-Max Scaler\
Slightly different from the Min-Max Scaling defined above:\
`
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min
`

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
mms.fit(df_final)
df_final_mms=mms.transform(df_final)

# 6. Data visualization

Data visualization is a wonderful way to get to know your data in order to plan a relevant analysis or find an appropriate machine learning application. [Matplotlib](https://matplotlib.org/) and [seaborn](https://seaborn.pydata.org/) are two canonical data visualization tools that you can use in Python. You will be learning more about it on Wednesday (module content can be found [here](https://github.com/neurodatascience/course-materials-2022/tree/main/Lectures/09-Intro_to_Data_Visualization). Below is but a taster for what lies ahead!

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#### Scatter plots

In [ ]:
plt.scatter(df_final["PSQI_BedTime"],df_final["TBV"]) # plot (x, y)
plt.ylabel("Total brain volume (mm3)") # label the y-axis
plt.xlabel("Bedtime (seconds)") # label the x-axis
plt.title("Total brain volume as a function of bed time") # title your graph

**Question 24:** Make a scatter plot of total brain volume (TBV) as a function of bed time, by gender.

In [ ]:
# SOLUTION
plt.scatter(df_final["PSQI_BedTime"][df_final["F"]==1.0],df_final["TBV"][df_final["F"]==1.0],color='turquoise')
    # plot female datapoints first, in turquoise
plt.scatter(df_final["PSQI_BedTime"][df_final["M"]==1.0],df_final["TBV"][df_final["M"]==1.0],color='coral')
    # plot female datapoints first, in coral
plt.ylabel("Total brain volume (mm3)")
plt.xlabel("Bedtime (seconds)")
plt.title("Total brain volume as a function of bed time")
plt.legend(['Females','Males']) # legend

#### Histogram plot

In [ ]:
sns.displot(df_final,x='TBV',kind='kde',fill=True) # plot your histogram
plt.ylabel("Density") # name y-axis
plt.xlabel("TBV (mm3)") # name x-axis
plt.title("Distribution of total brain volume (TBV) in our sample") # figure title
plt.gcf().set_size_inches(8, 5) # another way to adjust figure size

In [ ]:
sns.histplot(df_final,x='Str_Left',fill=True,color='orange')
    # first, plot the distribution of the left striatum in orange
sns.histplot(df_final,x='Thal_Left',fill=True,color='turquoise')
    # second, plot the distribution of the left thalamus volume in turquoise
plt.legend(['Left striatum','Left thalamus'])
plt.ylabel("Density")
plt.xlabel("Structure-specific volume (mm3)")
plt.title("Distribution of different structure volumes in our sample")
plt.gcf().set_size_inches(8, 5) # adjust figure size
plt.show()

**Question 25:** Generate two smoothed and superimposed histograms of bed times in subjects that are above and below 30 years.

In [ ]:
# SOLUTION
df_final["Over 25 years"]=df_final["Age"]>25
sns.displot(df_final,x='TBV',hue='Over 25 years',kind='kde',fill=True)
plt.ylabel("Density")
plt.xlabel("TBV (mm3)")
plt.title("Distribution of total brain volume (TBV) in our sample for people who are below and above 25 years of age")
plt.gcf().set_size_inches(14, 6) 

Notice that brains seem to shrink with age in our dataset... Let's look at the numbers:

In [ ]:
TBV_below_25=df_final['TBV'][df_final["Age"]<=25].mean()
TBV_above_25=df_final['TBV'][df_final["Age"]>25].mean()
print(f"Average TBV for people below or equal to 25 years of age: {round(TBV_below_25,0)} mm3") 
print(f"Average TBV for people above 25 years of age: {round(TBV_above_25,0)} mm3")

Interesting! However, does our sample actually have a similar amout of people who are below and above the age of 25? 

**Question 26:** Count how many people are <= 25 years, and how many people are > 25 yrs.

In [ ]:
subj_above_25=df_final[df_final["Age"]>25].shape[0]
subj_below_25=df_final[df_final["Age"]<=25].shape[0]
print(f"We have {subj_below_25} subjects below or at 25 years of age and {subj_above_25} subjects above 25 years of age.")

# 6. Handling 2D Images in Python
There are many different Python tools that you can use to convert 2D images into numpy arrays that you can then manipulate in various ways. For more information, check out [10 Python image manipulation tools](https://opensource.com/article/19/3/python-image-manipulation-tools). In the following (brief) tutorial, we will be using **s**ci**k**it image (**sk**image).

Locate your images

In [ ]:
!ls

In [ ]:
%cd HCP_2D_slices_MRI_data
!ls

Load an image into a numpy array

In [ ]:
from skimage import io
filename = 'HCP_102109_T1w_acpc_dc_restore_brain_t1_axial.png'
img=io.imread(filename)
print(f"Shape: {img.shape}")

**Question 27:** Why does is it a tensor with 4 "channels"? _Hint: this in a .png, not a .jpg, image!"

SOLUTION.
Each one of the 4 channels is a 2D matrix where the x and y coordinates correspond to a location in the 2D image and the actual number corresponds to either a red, green, blue or transparency value.

In [ ]:
import matplotlib.pyplot as plt  
plt.imshow(img)

Here is an example of a script that displays all of your axial images!\
_PS: The `glob` Python module can iterate through a directory using Bash!_

In [ ]:
import glob
images=[]
for file in glob.iglob('HCP*axial*png'):
    img=io.imread(file) 
    images.append(img)
fig = plt.figure(figsize=(15, 3.5))
rows = 1
columns = 5
for i in range(5):
    fig.add_subplot(rows, columns, i+1)
    img=images[i]
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Shape: {img.shape}",size=12) # height x width [x channel]
#fig.set_title("AXIAL")
plt.suptitle("AXIAL VIEW")

**Question 28:** Write a for loop to load & display all of your **sagittal** and **coronal** views of these images.

In [ ]:
# SOLUTION
images=[]
for file in glob.iglob('HCP*sagittal*png'):
    img=io.imread(file) 
    images.append(img)
fig = plt.figure(figsize=(15, 3.5))
rows = 1
columns = 5
for i in range(5):
    fig.add_subplot(rows, columns, i+1)
    img=images[i]
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Shape: {img.shape}",size=12) # height x width [x channel]
#fig.set_title("AXIAL")
plt.suptitle("SAGITTAL VIEW")

images=[]
for file in glob.iglob('HCP*coronal*png'):
    img=io.imread(file) 
    images.append(img)
fig = plt.figure(figsize=(15, 3.5))
rows = 1
columns = 5
for i in range(5):
    fig.add_subplot(rows, columns, i+1)
    img=images[i]
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Shape: {img.shape}",size=12) # height x width [x channel]
#fig.set_title("AXIAL")
plt.suptitle("CORONAL VIEW")